<a href="https://colab.research.google.com/github/caendeith/mlBTC/blob/main/mlBTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd

# Загрузка датасета
df = pd.read_csv('btcusd_1-min_data.csv')

# Преобразование Timestamp в нормальную дату
df['Date'] = pd.to_datetime(df['Timestamp'], unit='s', errors='coerce')

# Оставляем только Date и Close
df = df[['Date', 'Close']].copy()

# Сортируем по дате
df.sort_values(by='Date', inplace=True)
df.reset_index(drop=True, inplace=True)

print("Первые 5 строк:")
print(df.head())

Первые 5 строк:
                 Date  Close
0 2012-01-01 10:01:00   4.58
1 2012-01-01 10:02:00   4.58
2 2012-01-01 10:03:00   4.58
3 2012-01-01 10:04:00   4.58
4 2012-01-01 10:05:00   4.58


In [9]:
# Если есть несколько значений за день — берём последнее значение
df_daily = df.resample('D', on='Date').last().reset_index()
df_daily.columns = ['Date', 'Close']

print("После агрегации по дням:")
print(df_daily.tail())

После агрегации по дням:
           Date     Close
4905 2025-06-06  104393.0
4906 2025-06-07  105619.0
4907 2025-06-08  105792.0
4908 2025-06-09  110307.0
4909 2025-06-10  110110.0


In [10]:
import plotly.express as px

fig = px.line(df_daily, x='Date', y='Close', title='Цена Bitcoin во времени')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [11]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Нормализация
scaler = MinMaxScaler()
df_daily['Close_scaled'] = scaler.fit_transform(df_daily[['Close']])

# Создание последовательностей
def create_sequences(data, look_back=60):
    X, y = [], []
    for i in range(look_back, len(data)):
        X.append(data[i - look_back:i])
        y.append(data[i])
    return np.array(X), np.array(y)

LOOK_BACK = 60
X, y = create_sequences(df_daily['Close_scaled'].values, LOOK_BACK)

# Разделение на train/test
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print("Форма X_train:", X_train.shape)
print("Форма y_train:", y_train.shape)

Форма X_train: (3880, 60)
Форма y_train: (3880,)


In [12]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(units=50, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Обучаем модель
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



122/122 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 0.0060
Epoch 2/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 1.8151e-04
Epoch 3/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 1.6550e-04
Epoch 4/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - loss: 1.4863e-04
Epoch 5/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 1.5295e-04
Epoch 6/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 1.2724e-04
Epoch 7/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - loss: 1.0317e-04
Epoch 8/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 1.0620e-04
Epoch 9/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 9.4352e-05
Epoch 10/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 1.0129e-04


In [13]:
# Последние значения для прогноза
last_sequence = df_daily['Close_scaled'].values[-LOOK_BACK:]

forecast = []
for _ in range(7):
    prediction = model.predict(last_sequence[np.newaxis, :, np.newaxis], verbose=0)[0, 0]
    forecast.append(prediction)
    last_sequence = np.roll(last_sequence, -1)
    last_sequence[-1] = prediction

# Обратное преобразование
forecast_prices = scaler.inverse_transform(np.array(forecast).reshape(-1, 1)).flatten()
dates = pd.date_range(df_daily['Date'].iloc[-1], periods=8, freq='D')[1:]

print("\nПрогноз цены Bitcoin на неделю:")
for d, p in zip(dates, forecast_prices):
    print(f"{d.strftime('%Y-%m-%d')}: ${p:.2f}")


Прогноз цены Bitcoin на неделю:
2025-06-11: $105752.04
2025-06-12: $106045.96
2025-06-13: $105959.76
2025-06-14: $105581.89
2025-06-15: $105000.45
2025-06-16: $104281.41
2025-06-17: $103472.53


In [15]:
import plotly.graph_objects as go

fig = go.Figure()

# Реальные данные
fig.add_trace(go.Scatter(
    x=df_daily['Date'],
    y=df_daily['Close'],
    mode='lines',
    name='Реальная цена'
))

# Прогноз
fig.add_trace(go.Scatter(
    x=dates,
    y=forecast_prices,
    mode='lines+markers',
    name='Прогноз на 7 дней'
))

# Добавляем слайдер
fig.update_layout(
    title='Bitcoin — реальная цена и прогноз на неделю вперед',
    xaxis=dict(rangeslider=dict(visible=True), type='date'),
    template='plotly_white'
)

fig.show()